### **Export ComCam DeepCoadd Metadata to ECSV**

This notebook scans through the **ComCam deepCoadd FITS files** in the `data/` directory,  
parses their filenames to extract metadata, and writes the results into an **ECSV table**.  

The metadata columns are:

- `filename` – relative path to the FITS file (with `../../../dmu0/dmu0_ComCam` prefix)  
- `band` – filter band (e.g., `g`, `r`, etc.), extracted from the folder name `ComCam_band`  
- `patch` – patch index, parsed from the `i,j` folder (e.g. `2,2 → 22`)  
- `physical_filter` – filter name in `ComCam_band` format  
- `skymap` – fixed value: `lsst_cells_v1`  
- `tract` – tract ID, parsed from the folder name (e.g. `5063`)  

Finally, the script builds an **Astropy Table** and saves it as  
`example_comcam_deepcoadd.ecsv`. 

In [1]:
import glob
import os
from astropy.table import Table

rows = []

# Match all ComCam deepCoadd FITS files
for fits_file in sorted(glob.glob("data/*/*/*/*/*/deep_coadd_*fits")):
    try:
        # Normalize path
        rel_path = os.path.join("../../../dmu0/dmu0_ComCam", fits_file)

        # Split parts of the path
        parts = fits_file.split(os.sep)

        # Example parts: [..., "ComCam_g", "5063", "2,2", "deep_coadd_5063_22_g_...fits"]
        band = parts[-4].replace("ComCam_", "")
        tract = int(parts[-3])
        patch_folder = parts[-2]      # e.g. "2,2"
        patch = int(patch_folder.replace(",", ""))  # flatten "2,2" -> 22

        # Build columns
        physical_filter = f"ComCam_{band}"
        skymap = "lsst_cells_v1"

        rows.append([
            rel_path,
            band,
            patch,
            physical_filter,
            skymap,
            tract
        ])

    except Exception as e:
        print(f"⚠️ Problem parsing {fits_file}: {e}")

# Build Astropy table
t = Table(
    rows=rows,
    names=["filename", "band", "patch", "physical_filter", "skymap", "tract"]
)

# Save to ECSV
t.write("example_comcam_deepcoadd.ecsv", format="ascii.ecsv", overwrite=True)
print("✅ Wrote example_comcam_deepcoadd.ecsv with", len(rows), "rows")


✅ Wrote example_comcam_deepcoadd.ecsv with 6 rows
